In [6]:
import os
import re
import logging
from pathlib import Path

log_path = Path(os.path.join("download", "download.log"))

logger = logging.getLogger(__name__)
logging.basicConfig(filename=log_path, level=logging.INFO)

In [7]:
from file_read_backwards import FileReadBackwards


def purge_unfinished_file_from_logger() -> None:
    if not log_path.is_file() or os.stat(log_path).st_size == 0:
        return

    with FileReadBackwards(
        log_path, encoding="utf-8"
    ) as log_file:  # Read file from the back

        while True:
            line = str(log_file.readline())

            if line == "" or line == None:  # Skip end of or empty file
                continue
            elif (
                re.search("Finished", line) != None
            ):  # The last item finished. nothing to see here
                return
            else:
                last_download = re.search(
                    "INFO:.*?:Started - (.*)", line
                )  # Look for 'download started' in line

                if last_download != None:  # Save and finish
                    unfinished_file = f'{last_download[1].strip("\r\n").strip()}.mp4'

                    if os.path.exists(f'{last_download[1].strip("\r\n").strip()}.mp4'):
                        os.remove(unfinished_file)
                        print(
                            f"Removed unfinished file from last download: '{unfinished_file}'\n"
                        )

                    return

In [8]:
import ffmpeg


def convert_stream(path_and_name: str):
    logger.info(f"Started - {path_and_name}")

    print(f"Starting download of {path_and_name}")

    src = f"{path_and_name}.m3u8"
    dist = f"{path_and_name}.mp4"

    try:
        ffmpeg.input(src, protocol_whitelist="https,file,tls,tcp").output(
            dist, vcodec="libx265", vtag="hvc1"
        ).run()  # TODO: Progress counter

        # Set to finished so file isn't removed next run (overwrite == False)
        logger.info("Finished")
        # Safe to remove old file
        os.remove(src)

        return True
    except Exception:
        print(f"Download failed. Moving on.")

        if os.path.exists(dist):
            os.remove(dist)

        return False
    finally:
        # Close
        logger.info("Finished")

In [9]:
def main(overwrite: bool = True):
    # Remove file that didn't finish downloading in log-file
    if not overwrite:
        purge_unfinished_file_from_logger()

    files_to_convert: list[str] = []
    # TODO: Print files removed/ignored

    # Recursive find all in directory
    for folder in os.walk("download"):
        folder_path = folder[0]

        for file in folder[2]:
            if not file.endswith(".m3u8"):
                continue

            file_name = re.search("(.*).m3u8$", file)[1]
            path_and_name = os.path.join(folder_path, file_name)

            # The file already exists
            if os.path.exists(f"{path_and_name}.mp4"):
                # Should be overwritten
                if overwrite:
                    print(f"Overwriting old file: {path_and_name}")
                    os.remove(f"{path_and_name}.mp4")
                # Ignore new download request
                else:
                    print(f"File already exists: {path_and_name}")
                    os.remove(f"{path_and_name}.m3u8")
                    continue

            files_to_convert.append(path_and_name)

    print(f"Found {len(files_to_convert)} files to convert")

    failed_convertions: int = 0

    for index, file in enumerate(files_to_convert):
        if convert_stream(file):
            print(f"Converted {index + 1} of {len(files_to_convert)} files")
        else :
            failed_convertions += 1
            print (f"Failed to convert file {index + 1} of {len(files_to_convert)}")
        
    print(f"Finished {len(files_to_convert) - failed_convertions} of {len(files_to_convert)}{f". {failed_convertions} failed" if failed_convertions > 0 else ""}")

In [10]:
main()

print("Done")

Found 26 files to convert
Starting download of download\Seminars\2010.__ Poughkeepsie Koto-Ryū and Soujutsu\Iaijutsu (居合術)\0_Iai (居合)_1280x720
Download failed. Moving on.
Failed to convert file 1 of 26
Starting download of download\Seminars\2010.__ Poughkeepsie Koto-Ryū and Soujutsu\Iaijutsu (居合術)\2_Iai Aruki Ashi (居合歩き足)_1280x720
Download failed. Moving on.
Failed to convert file 2 of 26
Starting download of download\Seminars\2010.__ Poughkeepsie Koto-Ryū and Soujutsu\Iaijutsu (居合術)\3_And more Aruki Ashi ((居合歩き足)_1280x720
Download failed. Moving on.
Failed to convert file 3 of 26
Starting download of download\Seminars\2013.04 Syracuse - Sappō & Hanbo\Hanbo (半棒)\0_Sappō 殺法_1280x720
Download failed. Moving on.
Failed to convert file 4 of 26
Starting download of download\Seminars\2013.04 Syracuse - Sappō & Hanbo\Hanbo (半棒)\1_Tataki Otoshi 叩落_1280x720
Download failed. Moving on.
Failed to convert file 5 of 26
Starting download of download\Seminars\2013.04 Syracuse - Sappō & Hanbo\Hanbo (半

: 